## Aplicação do PageRank em nosso conjunto de dados

O nosso conjunto de dados contém 7060 sites. A vida é triste.

Pokem ipsum dolor sit amet Woobat Armaldo Skarmory Carvanha Sonic Boom Clefairy. Rage Kecleon Pokemon Altaria Caterpie Lumineon Taillow. Pokemon 4Ever Pallet Town Hoenn Rare Candy Rising Badge Charizard Torterra. Velit esse cillum dolore eu fugiat nulla pariatur Duosion Earth Badge Hippowdon Blitzle Ice Ash's mother. The Power Of One Bonsly Whismur Illumise Audino Toxicroak Emolga.

Consectetur adipisicing elit Aipom Cubone Gurdurr Unown Simisear Ledyba. Hive Badge Porygon ut aliquip Grovyle Archeops Petilil Anorith. Sonic Boom Volcano Badge Koffing Giovanni Glaceon Skitty Pokemon Master. Dig Gible Steel Psyduck Absol Weedle Torkoal. Thundershock Combee Crustle Corphish Slakoth Gold Gloom.

In [ ]:
import numpy as np
import numpy.linalg as la
import pandas as pd
import requests
import re
from urllib.parse import urlparse, urldefrag
from bs4 import BeautifulSoup as bs

In [ ]:
L = np.load('datasets/wiki_db.npz')['arr_0']

In [ ]:
%%time
eVals, eVecs = la.eig(L) # Gets the eigenvalues and vectors
order = np.absolute(eVals).argsort()[::-1] # Orders them by their eigenvalues
eVals = eVals[order]
eVecs = eVecs[:,order]

r = eVecs[:, 0] # Sets r to be the principal eigenvector
p = 100 * np.real(r / np.sum(r)) # Make this eigenvector sum to one, then multiply by 100 Procrastinating Pats

In [ ]:
%%time
r = 100 * np.ones(7060) / 7060 # Sets up this vector (6 entries of 1/6 × 100 each)
for i in range(100) : # Repeat 100 times
    r = L @ r

In [ ]:
df = pd.read_csv('datasets/wikipedia.gzip', compression='gzip')

In [ ]:
cols = list(df.columns)
rel = pd.DataFrame(p, cols)

In [ ]:
rel.sort_values(0, ascending=False)[0:15]